<a href="https://colab.research.google.com/github/chiyeon01/AI-Study/blob/main/Oxpet_Dataset2UltralyticsYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov3.git

Cloning into 'yolov3'...
remote: Enumerating objects: 11693, done.
remote: Counting objects: 100% (1615/1615), done.
remote: Compressing objects: 100% (746/746), done.
remote: Total 11693 (delta 1149), reused 1224 (delta 868), pack-reused 10078 (from 1)
Receiving objects: 100% (11693/11693), 10.31 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (7948/7948), done.


In [2]:
!cd yolov3;pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 760.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00


In [3]:
!pwd

/content


### Dataset는 Oxford Pet Dataset 이용
* Dataset download : [https://www.robots.ox.ac.uk/~vgg/data/pets/](https://www.robots.ox.ac.uk/~vgg/data/pets/)

In [4]:
!mkdir /content/oxford_dataset

In [5]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz

--2024-08-22 13:05:08--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2024-08-22 13:05:08--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  25.6MB/s    in 31s     

2024-08-22 13:05:40 (24.1 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2024-08-22 13:05:40--  https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.o

In [6]:
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

In [7]:
# oxford dataset을 ultralytics yolo 형식으로 만들기(images <-> labels)

!cd oxford_dataset; mkdir ./images; mkdir ./labels
!cd oxford_dataset/images; mkdir ./train; mkdir ./val
!cd oxford_dataset/labels; mkdir ./train; mkdir ./val

In [8]:
!cd ~
!pwd

/content


In [1]:
# df형식으로 바꾸는 이유는 한번에 itteration하기 위해

import pandas as pd

oxford_dataset = pd.read_csv('/content/data/annotations/trainval.txt', sep=' ', names=['file_name', 'class_id', 'etc1', 'etc2'], header=None)
oxford_dataset.head()

,file_name,class_id,etc1,etc2
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [2]:
# dataframe에 annotation file path / image file path 정보를 넣어줌.
# 이중에 annotation file path가 실제로 존재하지 않지만 txt에 넣어져 있는 경우도 있으므로 remove_no_ann_df를 이용해 제거해줌.
# 그렇게 만든 dataframe을 train/test용으로 나누어줌.

import os
from sklearn.model_selection import train_test_split

img_prefix = '/content/data/images/'
ann_prefix = '/content/data/annotations/xmls/'

def make_train_val_dataframe(df, img_prefix, ann_prefix, test_size=0.1):

  df['class_name'] = df['file_name'].apply(lambda x:x[:x.rfind('_')])
  df['image_file_path'] = img_prefix + df['file_name'] + '.jpg'
  df['annotation_file_path'] = ann_prefix + df['file_name'] + '.xml'

  df = remove_no_ann_df(df)
  train_df, test_df = train_test_split(df,test_size=test_size, stratify=df['class_id'])
  return train_df, test_df

def remove_no_ann_df(df):
  remove_df_idx = []

  for index, d in df.iterrows():
    ann_file = d['annotation_file_path']
    img_file = d['image_file_path']
    if not os.path.exists(ann_file):
      print('##### index:', index, ann_file, '가 존재하지 않아서 Dataframe에서 삭제함')
      remove_df_idx.append(index)
    elif not os.path.exists(img_file):
      print('##### index:', index, img_file, '가 존재하지 않아서 Dataframe에서 삭제함')
      remove_df_idx.append(index)
  df = df.drop(remove_df_idx, axis=0, inplace=False)
  return df

In [3]:
train_df, test_df = make_train_val_dataframe(oxford_dataset, img_prefix, ann_prefix)

##### index: 4 /content/data/annotations/xmls/Abyssinian_104.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 262 /content/data/annotations/xmls/Bengal_111.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 1456 /content/data/annotations/xmls/samoyed_10.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2128 /content/data/annotations/xmls/Bengal_175.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2395 /content/data/annotations/xmls/Egyptian_Mau_14.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2402 /content/data/annotations/xmls/Egyptian_Mau_156.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 2427 /content/data/annotations/xmls/Egyptian_Mau_186.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 3177 /content/data/annotations/xmls/Ragdoll_199.xml 가 존재하지 않아서 Dataframe에서 삭제함
##### index: 3246 /content/data/annotations/xmls/saint_bernard_15.xml 가 존재하지 않아서 Dataframe에서 삭제함


In [4]:
train_df.shape, test_df.shape

((3303, 7), (368, 7))

In [5]:
train_df.head()

,file_name,class_id,etc1,etc2,class_name,image_file_path,annotation_file_path
258,Bengal_108,6,1,2,Bengal,/content/data/images/Bengal_108.jpg,/content/data/annotations/xmls/Bengal_108.xml
1265,pug_119,26,2,18,pug,/content/data/images/pug_119.jpg,/content/data/annotations/xmls/pug_119.xml
1817,yorkshire_terrier_11,37,2,25,yorkshire_terrier,/content/data/images/yorkshire_terrier_11.jpg,/content/data/annotations/xmls/yorkshire_terri...
620,english_cocker_spaniel_122,13,2,7,english_cocker_spaniel,/content/data/images/english_cocker_spaniel_12...,/content/data/annotations/xmls/english_cocker_...
1689,Sphynx_152,34,1,12,Sphynx,/content/data/images/Sphynx_152.jpg,/content/data/annotations/xmls/Sphynx_152.xml


In [6]:
import xml.etree.ElementTree as ET

def xml2txt(df, type):
  for index, d in df.iterrows():
    output_path_txt = '/content/oxford_dataset/labels/' + type + d['file_name'] + '.txt'
    with open(output_path_txt, 'w') as write_path:
        ann_file_path = d['annotation_file_path']
        class_id = d['class_id']
        tree = ET.parse(ann_file_path)
        root = tree.getroot()
        size_root = root.find('size')
        cols = int(size_root.find('width').text)
        rows = int(size_root.find('height').text)

        for ob in root.findall('object'):
          box = ob.find('bndbox')
          x1 = int(box.find('xmin').text)
          y1 = int(box.find('ymin').text)
          x2 = int(box.find('xmax').text)
          y2 = int(box.find('ymax').text)

          center_x = (x1 + y1)/cols
          center_y = (x2 + y2)/rows
          width = (x2 - x1)/cols
          height = (y2 - y1)/rows
          input_text = "{} {:.4f} {:.4f} {:.4f} {:.4f}".format(class_id, center_x, center_y, width, height)
          write_path.write(input_text + '\n')

def img2img(df, output_path):
  for index, d in df.iterrows():
    image_file_path = d['image_file_path']
    !cp image_file_path output_path

In [7]:
output_path_txt_train = '/content/oxford_dataset/labels/train'
output_path_txt_val = '/content/oxford_dataset/labels/val'
output_path_train = '/content/oxford_dataset/images/train'
output_path_val = '/content/oxford_dataset/images/val'

In [ ]:
xml2txt(train_df, 'train')
xml2txt(test_df, 'val')
img2img(train_df, output_path_train)
img2img(test_df, output_path_val)

cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_file_path': No such file or directory
cp: cannot stat 'image_f

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> yolov3
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached yolov3
hint: 
hint: See "git help submodule" for more information.
